# Add user and description to JSON



In [1]:
import io
import os
import json

In [2]:
class JsonLocationsHelper:

    def __init__(self, path):
        self.path = path
        self.classes = [x for x in os.listdir(path) if "." not in x]
        self.locations = None
        self.country_mapping = self.load_country_mapping()
        self.all_image_infos = {}
        self.load_all_locations()
        print(str(self.classes))

    def load_country_mapping(self):
        file_path = os.path.join(self.path, "country-alpha-2-mapping.json")

        with open(file_path, "r") as f:
            content = json.load(f)

        print("loaded {} country to alpha 2 code mappings.".format(len(content)))
        return content

    def load_all_locations(self):
        if self.locations is None:
            self.locations = {}
            for label in self.classes:
                self.locations[label.lower()] = self.load_locations_for_label(label)
        print("loaded {} labels".format(len(self.all_image_infos)))
        return self.locations

    def load_locations_for_label(self, label):
        print("load_locations_for_label > label: {}".format(label))
        label_path = os.path.join(self.path, label)
        locations = {}
        files = [x for x in os.listdir(label_path) if x.endswith("json")]
        i = 0
        for file in files:
            json_file_path = os.path.join(label_path, file)
            with open(json_file_path, "r") as f:
                image_id = os.path.splitext(file)[0]
                content = json.load(f)
                if content is not None:
                    country = content["attributes"]["table"]["country"]
                    id_infos = content["attributes"]["table"]
                    id_infos['id'] = image_id
                    if country is not None:

                        # get alpha 2 code
                        alpha_code = self.country_mapping.get(country.strip(), country.strip())
                        id_infos['country'] = alpha_code
                        if alpha_code not in locations:
                            locations[alpha_code] = []

                        i = i + 1
                        locations[alpha_code].append(id_infos)
                    self.all_image_infos[image_id] = id_infos
        print("load_locations_for_label > found {} locations".format(i))
        return locations

    def get_locations_for_label(self, label):
        if self.locations is None:
            self.load_all_locations()

        locations = self.locations.get(label.lower())
        return locations

    def get_all_labels(self):
        return self.classes

    def get_information_for_id(self, id):
        return self.all_image_infos[id]

    def get_all_ids(self):
        return list(self.all_image_infos.keys())


In [3]:
json_loader = JsonLocationsHelper("json/")
labels = json_loader.get_all_labels()

loaded 143 country to alpha 2 code mappings.
load_locations_for_label > label: Forest
load_locations_for_label > found 185 locations
load_locations_for_label > label: Glacier
load_locations_for_label > found 260 locations
load_locations_for_label > label: Galaxy
load_locations_for_label > found 184 locations
load_locations_for_label > label: Ruins
load_locations_for_label > found 210 locations
load_locations_for_label > label: Jungle
load_locations_for_label > found 141 locations
load_locations_for_label > label: Waterfalls
load_locations_for_label > found 215 locations
load_locations_for_label > label: Beach
load_locations_for_label > found 187 locations
load_locations_for_label > label: Desert
load_locations_for_label > found 189 locations
load_locations_for_label > label: Mountains
load_locations_for_label > found 223 locations
load_locations_for_label > label: Sea
load_locations_for_label > found 183 locations
loaded 1997 labels
['Forest', 'Glacier', 'Galaxy', 'Ruins', 'Jungle', 'W

In [4]:
dummy_user = '''
{
    "id": "vbNwzjKGx-0",
    "updated_at": "2020-03-23T01:58:18-04:00",
    "username": "lamerbrain",
    "name": "Matt Lamers",
    "first_name": "Matt",
    "last_name": "Lamers",
    "twitter_username": null,
    "portfolio_url": null,
    "bio": "Digital advertising professional who has recently picked up photography as a hobby on the side.  Really into long exposures and landscapes.",
    "location": null,
    "links": {
        "self": "https://api.unsplash.com/users/lamerbrain",
        "html": "https://unsplash.com/@lamerbrain",
        "photos": "https://api.unsplash.com/users/lamerbrain/photos",
        "likes": "https://api.unsplash.com/users/lamerbrain/likes",
        "portfolio": "https://api.unsplash.com/users/lamerbrain/portfolio",
        "following": "https://api.unsplash.com/users/lamerbrain/following",
        "followers": "https://api.unsplash.com/users/lamerbrain/followers"
    },
    "profile_image": {
        "small": "https://images.unsplash.com/profile-fb-1489023170-de3477ccf5dd.jpg?ixlib=rb-1.2.1&q=80&fm=jpg&crop=faces&cs=tinysrgb&fit=crop&h=32&w=32",
        "medium": "https://images.unsplash.com/profile-fb-1489023170-de3477ccf5dd.jpg?ixlib=rb-1.2.1&q=80&fm=jpg&crop=faces&cs=tinysrgb&fit=crop&h=64&w=64",
        "large": "https://images.unsplash.com/profile-fb-1489023170-de3477ccf5dd.jpg?ixlib=rb-1.2.1&q=80&fm=jpg&crop=faces&cs=tinysrgb&fit=crop&h=128&w=128"
    },
    "instagram_username": "lamerbrain",
    "total_collections": 0,
    "total_likes": 3,
    "total_photos": 80,
    "accepted_tos": false
}
'''
dummy_description = 'Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. '

In [6]:
def modify_json(path, label):
        print("load_locations_for_label > label: {}".format(label))
        label_path = os.path.join(path, label)
        locations = {}
        files = [x for x in os.listdir(label_path) if x.endswith("json")]
        i = 0
        for file in files:
            json_file_path = os.path.join(label_path, file)
            print(json_file_path)
            with open(json_file_path, "r") as f:
                image_id = os.path.splitext(file)[0]
                content = json.load(f)
                if content is not None:
                    content['user'] = json.load(io.StringIO(dummy_user))
                    content['description'] = dummy_description
            if content is not None: 
                with open(json_file_path, "w") as out:        
                    json.dump(content, out, ensure_ascii=False, indent=4)
                    print("wrote to {}".format(json_file_path))
                    
                    

In [7]:
for l in ['Forest', 'Glacier', 'Galaxy', 'Ruins', 'Jungle', 'Waterfalls', 'Beach', 'Desert', 'Mountains', 'Sea']:
    modify_json('json/', l)

load_locations_for_label > label: Forest
json/Forest/LZpzMEY0pXI.json
wrote to json/Forest/LZpzMEY0pXI.json
json/Forest/Km5JGMW4VPM.json
wrote to json/Forest/Km5JGMW4VPM.json
json/Forest/wsFzXgWMlLA.json
json/Forest/IZdwSeYBoRk.json
json/Forest/t-3f5RS-JGI.json
wrote to json/Forest/t-3f5RS-JGI.json
json/Forest/IUFcISwXiH8.json
json/Forest/GzUhyUoDFRI.json
json/Forest/3ytjETpQMNY.json
wrote to json/Forest/3ytjETpQMNY.json
json/Forest/6_azJTdV9xg.json
wrote to json/Forest/6_azJTdV9xg.json
json/Forest/y_ykvs7Z6fo.json
wrote to json/Forest/y_ykvs7Z6fo.json
json/Forest/jGbSl2lXcoU.json
wrote to json/Forest/jGbSl2lXcoU.json
json/Forest/xY_6ZENqcfo.json
wrote to json/Forest/xY_6ZENqcfo.json
json/Forest/fAZSHN2fIGc.json
wrote to json/Forest/fAZSHN2fIGc.json
json/Forest/_yWe_pzIj2E.json
wrote to json/Forest/_yWe_pzIj2E.json
json/Forest/ot-A6eBTJG4.json
json/Forest/pCU3zt9OEn0.json
json/Forest/dYshDcTI1Js.json
json/Forest/Ok7VJ-uFkyM.json
wrote to json/Forest/Ok7VJ-uFkyM.json
json/Forest/WvlS1yW

wrote to json/Glacier/djbaKcyBZfY.json
json/Glacier/bwD3GLrV4pY.json
wrote to json/Glacier/bwD3GLrV4pY.json
json/Glacier/cdZh5dQ-muE.json
wrote to json/Glacier/cdZh5dQ-muE.json
json/Glacier/AY3_mI0TW14.json
json/Glacier/UIWXv4kXCQo.json
wrote to json/Glacier/UIWXv4kXCQo.json
json/Glacier/HzSLh5zV42s.json
wrote to json/Glacier/HzSLh5zV42s.json
json/Glacier/DyX93yan1nE.json
json/Glacier/vZ1JAXUO3-0.json
wrote to json/Glacier/vZ1JAXUO3-0.json
json/Glacier/CLr3GuO7OVU.json
wrote to json/Glacier/CLr3GuO7OVU.json
json/Glacier/NXg7Be3Fk7A.json
wrote to json/Glacier/NXg7Be3Fk7A.json
json/Glacier/NoLg1Cg4t2c.json
json/Glacier/xEh4hvxRKXM.json
wrote to json/Glacier/xEh4hvxRKXM.json
json/Glacier/oO8hkJCwyv4.json
wrote to json/Glacier/oO8hkJCwyv4.json
json/Glacier/nrRuOmeIiOI.json
wrote to json/Glacier/nrRuOmeIiOI.json
json/Glacier/JACEkXaAay0.json
json/Glacier/HpJUwdFDWhk.json
json/Glacier/sQgxCIfYxzc.json
wrote to json/Glacier/sQgxCIfYxzc.json
json/Glacier/W4rUHIw3pd0.json
wrote to json/Glacier/

wrote to json/Galaxy/1t6q90s0Zkk.json
json/Galaxy/q--1RyVkoD4.json
json/Galaxy/_vWMEPIlg1Y.json
wrote to json/Galaxy/_vWMEPIlg1Y.json
json/Galaxy/1JHzqk5oTy8.json
wrote to json/Galaxy/1JHzqk5oTy8.json
json/Galaxy/NeCNxRwobes.json
wrote to json/Galaxy/NeCNxRwobes.json
json/Galaxy/eorb_1PkoB8.json
wrote to json/Galaxy/eorb_1PkoB8.json
json/Galaxy/0s2eg7cBLQ8.json
json/Galaxy/0o_GEzyargo.json
json/Galaxy/8riFHUtqoXM.json
wrote to json/Galaxy/8riFHUtqoXM.json
json/Galaxy/qy8XDguG9bw.json
wrote to json/Galaxy/qy8XDguG9bw.json
json/Galaxy/6lH1_oSgjX0.json
wrote to json/Galaxy/6lH1_oSgjX0.json
json/Galaxy/zRwwsLW5Oh4.json
wrote to json/Galaxy/zRwwsLW5Oh4.json
json/Galaxy/TlW_c7i2n58.json
json/Galaxy/eeiAnugy2Hs.json
json/Galaxy/OOBtVJrDZUw.json
wrote to json/Galaxy/OOBtVJrDZUw.json
json/Galaxy/-tXcEuihgzE.json
wrote to json/Galaxy/-tXcEuihgzE.json
json/Galaxy/Wroo94nLobM.json
json/Galaxy/mA6ZjlOmnZw.json
wrote to json/Galaxy/mA6ZjlOmnZw.json
json/Galaxy/WWOKTQKym4k.json
wrote to json/Galaxy/W

wrote to json/Ruins/6HLT9w7koFM.json
json/Ruins/Osd4ngHD4kM.json
wrote to json/Ruins/Osd4ngHD4kM.json
json/Ruins/V8AnLRYWsM8.json
wrote to json/Ruins/V8AnLRYWsM8.json
json/Ruins/ippIhSrimC8.json
wrote to json/Ruins/ippIhSrimC8.json
json/Ruins/5pUv8FdsAEY.json
wrote to json/Ruins/5pUv8FdsAEY.json
json/Ruins/uh0u8OH4zuE.json
wrote to json/Ruins/uh0u8OH4zuE.json
json/Ruins/LzhKgc9sBKg.json
wrote to json/Ruins/LzhKgc9sBKg.json
json/Ruins/8SAxrmRbFOc.json
json/Ruins/Kswnnx95_YU.json
wrote to json/Ruins/Kswnnx95_YU.json
json/Ruins/g_ssp9teiSA.json
wrote to json/Ruins/g_ssp9teiSA.json
json/Ruins/8zJSg57TBBA.json
json/Ruins/nDkWdmAuJ-I.json
json/Ruins/rHi-KR_NA24.json
wrote to json/Ruins/rHi-KR_NA24.json
json/Ruins/8vfoNRBysD4.json
wrote to json/Ruins/8vfoNRBysD4.json
json/Ruins/QrEx5vIitfY.json
wrote to json/Ruins/QrEx5vIitfY.json
json/Ruins/-ILZVY1vG_k.json
wrote to json/Ruins/-ILZVY1vG_k.json
json/Ruins/Zi7hgVlMnqQ.json
wrote to json/Ruins/Zi7hgVlMnqQ.json
json/Ruins/aF-q01CL_Mc.json
wrote 

wrote to json/Jungle/fG5jun4bYBQ.json
json/Jungle/eP2885MMLmo.json
json/Jungle/fMkqohr7Ojs.json
wrote to json/Jungle/fMkqohr7Ojs.json
json/Jungle/LZ-YUm7Mqd4.json
wrote to json/Jungle/LZ-YUm7Mqd4.json
json/Jungle/rJ5vHo8gr2U.json
json/Jungle/Nk088TzYUlI.json
wrote to json/Jungle/Nk088TzYUlI.json
json/Jungle/sMQiL_2v4vs.json
json/Jungle/MjDHwzk8zR0.json
wrote to json/Jungle/MjDHwzk8zR0.json
json/Jungle/UEjIW0GNtlU.json
json/Jungle/LNSIGPuZXIg.json
wrote to json/Jungle/LNSIGPuZXIg.json
json/Jungle/CHtHMPRcVr8.json
wrote to json/Jungle/CHtHMPRcVr8.json
json/Jungle/9G1kXjqOq4U.json
json/Jungle/p4orVxNl5Ko.json
json/Jungle/3WJiY5vBLT0.json
wrote to json/Jungle/3WJiY5vBLT0.json
json/Jungle/Gi9hYOGKFKE.json
json/Jungle/WDEeYo-nPpU.json
wrote to json/Jungle/WDEeYo-nPpU.json
json/Jungle/p6vtBagDeFk.json
wrote to json/Jungle/p6vtBagDeFk.json
json/Jungle/A4dcSmJsr3Q.json
json/Jungle/-RQ8tGcn2EY.json
json/Jungle/NTyBbu66_SI.json
wrote to json/Jungle/NTyBbu66_SI.json
json/Jungle/31aqlSiMhpw.json
wr

wrote to json/Beach/qZoMsoX527I.json
json/Beach/a6B87dG9GeY.json
wrote to json/Beach/a6B87dG9GeY.json
json/Beach/pZyiD88Z8Ts.json
wrote to json/Beach/pZyiD88Z8Ts.json
json/Beach/L_hK813fu9k.json
wrote to json/Beach/L_hK813fu9k.json
json/Beach/H764P2--L9w.json
json/Beach/HzXhY9Wgzbw.json
wrote to json/Beach/HzXhY9Wgzbw.json
json/Beach/7OPzL-Y0O5o.json
wrote to json/Beach/7OPzL-Y0O5o.json
json/Beach/nJxNoxWzF7Y.json
wrote to json/Beach/nJxNoxWzF7Y.json
json/Beach/Ee5NWvWhuWM.json
wrote to json/Beach/Ee5NWvWhuWM.json
json/Beach/jsiOxtBM53w.json
wrote to json/Beach/jsiOxtBM53w.json
json/Beach/8BH0YamKVAs.json
wrote to json/Beach/8BH0YamKVAs.json
json/Beach/iH9iF9DfDuw.json
wrote to json/Beach/iH9iF9DfDuw.json
json/Beach/04paG2PxKGs.json
json/Beach/9Jq5UvyXr8A.json
wrote to json/Beach/9Jq5UvyXr8A.json
json/Beach/N61-GGQAp6Y.json
wrote to json/Beach/N61-GGQAp6Y.json
json/Beach/F-xmidvSkHc.json
wrote to json/Beach/F-xmidvSkHc.json
json/Beach/i93OgkKVHPI.json
wrote to json/Beach/i93OgkKVHPI.js

json/Desert/G-774VBS6Fo.json
json/Desert/jRYBR41lx3Y.json
wrote to json/Desert/jRYBR41lx3Y.json
json/Desert/fRYbWNQKdP8.json
wrote to json/Desert/fRYbWNQKdP8.json
json/Desert/Iw2_QHOaSy0.json
wrote to json/Desert/Iw2_QHOaSy0.json
json/Desert/JAv2mR6oE_w.json
json/Desert/1oDfOmXHpgQ.json
wrote to json/Desert/1oDfOmXHpgQ.json
json/Desert/lGqLLvaz_Ac.json
wrote to json/Desert/lGqLLvaz_Ac.json
json/Desert/KNxk9hVHM3Q.json
json/Desert/yNGQ830uFB4.json
wrote to json/Desert/yNGQ830uFB4.json
json/Desert/yWbxfKurMH0.json
wrote to json/Desert/yWbxfKurMH0.json
json/Desert/XpAFtjJEMeU.json
wrote to json/Desert/XpAFtjJEMeU.json
json/Desert/6E8rH7lKOMA.json
json/Desert/zDRv_mCdEjM.json
json/Desert/yL9cQW-P72Q.json
wrote to json/Desert/yL9cQW-P72Q.json
json/Desert/fzrplAw24xI.json
wrote to json/Desert/fzrplAw24xI.json
json/Desert/xq3pyFAJoSU.json
wrote to json/Desert/xq3pyFAJoSU.json
json/Desert/Mw00TR34O2c.json
wrote to json/Desert/Mw00TR34O2c.json
json/Desert/hniuowbXbBY.json
wrote to json/Desert/h

wrote to json/Mountains/vAqmcvSMWMU.json
json/Mountains/YV2Nchv9pd4.json
wrote to json/Mountains/YV2Nchv9pd4.json
json/Mountains/zs_hIyXb_Ns.json
json/Mountains/quwi4pXwOfw.json
wrote to json/Mountains/quwi4pXwOfw.json
json/Mountains/2Qw-6ICrNqQ.json
wrote to json/Mountains/2Qw-6ICrNqQ.json
json/Mountains/8baIfu7bKsE.json
json/Mountains/uunwfiSLZQU.json
wrote to json/Mountains/uunwfiSLZQU.json
json/Mountains/nKpoNQx2R5g.json
wrote to json/Mountains/nKpoNQx2R5g.json
json/Mountains/UWebmzDCWjI.json
wrote to json/Mountains/UWebmzDCWjI.json
json/Mountains/-h-_S37Eu_0.json
wrote to json/Mountains/-h-_S37Eu_0.json
json/Mountains/EA7uXUeAmjU.json
wrote to json/Mountains/EA7uXUeAmjU.json
json/Mountains/9Vjiir72sOk.json
wrote to json/Mountains/9Vjiir72sOk.json
json/Mountains/PTkB0AvR4M0.json
wrote to json/Mountains/PTkB0AvR4M0.json
json/Mountains/g30P1zcOzXo.json
wrote to json/Mountains/g30P1zcOzXo.json
json/Mountains/h1y8LqxFn0o.json
wrote to json/Mountains/h1y8LqxFn0o.json
json/Mountains/x2NS